# Import & Setup

In [136]:
import sys
sys.path.append(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/My-Package")

import pandas as pd
import numpy as np
from myfunctions import clean_path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px



In [137]:
crosstabs = pd.read_csv(r"Crosstabs by Race.csv").rename(columns={'Unnamed: 0':'Race'}).set_index('Race')

In [138]:
crosstabs.rename(columns={'Stops weighted by # tickets':'Stops weighted by \n \xa0 # tickets'})

,MC Pop.,Accidents,Stops,Stops X # Tickets,Citations,Warnings,Arrests,Searches,Prob. Cause
Race,,,,,,,,,
ASIAN,0.156,0.045455,0.054001,0.04612,0.048206,0.055763,0.026432,0.030641,0.026667
BLACK,0.201,0.285124,0.319746,0.35148,0.310419,0.329566,0.383260,0.473538,0.620000
HISPANIC,0.201,0.322314,0.226028,0.25780,0.266969,0.204286,0.303965,0.253482,0.153333
NATIVE AMERICAN,0.000,0.000000,0.000673,0.00048,0.000865,0.000640,0.004405,0.002786,0.006667
OTHER,0.013,0.039256,0.070307,0.05804,0.057933,0.075168,0.022026,0.016713,0.026667
WHITE,0.429,0.307851,0.329245,0.28608,0.315607,0.334577,0.259912,0.222841,0.166667


# Visualize

In [139]:
go.Histogram(x=crosstabs.loc['ASIAN'], nbinsx=4)

Histogram({
    'nbinsx': 4,
    'x': array([0.156     , 0.04545455, 0.0540015 , 0.04612   , 0.04820579, 0.05576287,
                0.02643172, 0.03064067, 0.02666667])
})

In [140]:
fig = go.Figure()
for race in crosstabs.index:
    fig.add_trace(go.Bar(
        name=race,
        y=crosstabs.loc[race],
        x=crosstabs.columns,
        offset=0,
        # customdata=np.transpose([labels, widths*data[race]]),
        # texttemplate="%{y} x %{width} =<br>%{customdata[1]}",
        # textposition="inside",
        textangle=0,
        # textfont_color="white",
        # hovertemplate="<br>".join([
        #     "label: %{crosstabs.columns}",
        #     "width: %{width}",
        #     "height: %{y}",
        #     "area: %{customdata[1]}",
        # ])
    ))

# fig.update_xaxes(
#     tickvals=np.cumsum(widths)-widths/2,
#     ticktext= ["%s<br>%d" % (l, w) for l, w in zip(labels, widths)]
# )

# fig.update_xaxes(range=[0,100])
# fig.update_yaxes(range=[0,100])

fig.update_xaxes(
        tickangle = 45,
        title = 'Traffic Outcome'
        )

fig.update_layout(
    title_text="Race X Traffic Outcomes",
    barmode="stack",
    uniformtext=dict(mode="hide", minsize=10),
    # hovermode="y"
)
# fig.update_layout()


# Multiple Bar Charts

In [141]:
crosstabs

,MC Pop.,Accidents,Stops,Stops X # Tickets,Citations,Warnings,Arrests,Searches,Prob. Cause
Race,,,,,,,,,
ASIAN,0.156,0.045455,0.054001,0.04612,0.048206,0.055763,0.026432,0.030641,0.026667
BLACK,0.201,0.285124,0.319746,0.35148,0.310419,0.329566,0.383260,0.473538,0.620000
HISPANIC,0.201,0.322314,0.226028,0.25780,0.266969,0.204286,0.303965,0.253482,0.153333
NATIVE AMERICAN,0.000,0.000000,0.000673,0.00048,0.000865,0.000640,0.004405,0.002786,0.006667
OTHER,0.013,0.039256,0.070307,0.05804,0.057933,0.075168,0.022026,0.016713,0.026667
WHITE,0.429,0.307851,0.329245,0.28608,0.315607,0.334577,0.259912,0.222841,0.166667


In [142]:
color_dict = {race:px.colors.qualitative.Dark2[i] for i, race in enumerate(crosstabs.index)}

In [152]:
fig = make_subplots(rows=2,
                    cols=3,
                    subplot_titles=crosstabs.index,
                    horizontal_spacing = 0.05, 
                    vertical_spacing=.2, shared_yaxes=True)#, shared_yaxes=True)

for i, race in enumerate(crosstabs.index):
    race_row = int(1 + np.floor(i/3))
    race_col = 1 + i%3
    fig.add_trace(go.Bar(x=crosstabs.columns,
                         y=crosstabs.loc[race],
                         opacity=.65,
                         name=race, 
                         marker_color=['black', 'black'] + [color_dict[race]]*(len(crosstabs.columns)-2)),
                  row=race_row,
                  col=race_col)
    fig.add_shape(go.layout.Shape(type="line", 
                                  yref="paper",
                                  xref="x",
                                  x0=-.5,
                                  x1=len(crosstabs.columns),
                                  y0=crosstabs.loc[race, "MC Pop."],
                                  y1=crosstabs.loc[race, "MC Pop."],
                                  line=dict(color='red', width=.5, dash='dash')),
                  row=race_row,
                  col=race_col)
fig.update_layout(autosize=False,
                  width=1200,
                  height=800,
                  showlegend=False, title='Traffic Stop Statistics by Race',
                  title_x=0.5)
fig.show()
fig.write_html("Traffic Stop Statistics by Race - Interactive.html")

# Long Crosstbas approach

In [144]:
long_crosstabs = crosstabs.T.reset_index().rename(columns={'index':'Traffic Var'}).melt(id_vars=['Traffic Var'], value_vars = ['ASIAN', 'BLACK', 'HISPANIC', 'NATIVE AMERICAN', 'OTHER', 'WHITE'])

* make MC pop "special" (so ppl know it's the reference)

In [146]:
fig=px.bar(long_crosstabs, x='Traffic Var', y='value', color='Race', opacity=.5, color_discrete_sequence=px.colors.qualitative.Dark2)
fig.show()